In [153]:
#爬中基协产品数据
from urllib.parse import urlencode
import requests
import json
import datetime
base_url = 'http://gs.amac.org.cn/amac-infodisc/api/pof/fund?'        # 请求 url 所携带参数的前面部分
fund_base_url = 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/'   # 基金 url 前面的部分
lis_json = []                                                              # 存放 json 数据

headers = {                                                      # 请求头
	'Accept': 'application/json, text/javascript, */*; q=0.01',
	'Accept-Encoding': 'gzip, deflate',
	'Accept-Language': 'zh-CN,zh;q=0.9',
	'Cache-Control':'max-age=0',
	'Connection': 'keep-alive',
	'Content-Length': '2',
	'Content-Type': 'application/json',
	'Host': 'gs.amac.org.cn',
	'Origin': 'http://gs.amac.org.cn',
	'Referer': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/index.html',
	'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
	'X-Requested-With': 'XMLHttpRequest'
}

def get_page(page):
    '''爬取第xx页信息'''
    # url 携带参数，设置了每页显示 100 条信息
    params = {                                                  
        'rand': 0.3248183083707361,
        'page': page,
        'size': 100,
    }
    url = base_url + urlencode(params)
    try:
        r = requests.post(url,headers=headers,data=json.dumps({}))      # post 方法，将 Request Payload 的信息转换成 json 格式然后提交
        if r.status_code == 200:
            return r.json()
    except requests.ConnectionError as e:
        print('Error',e.args)

def parse_page(r_json):
    if r_json:
        items = r_json.get('content')
        for item in items:
            info = {}
            info['基金名称'] = item.get('fundName')
            info['私募基金管理人名称'] = item.get('managerName')
            info['托管人名称'] = item.get('mandatorName')
            establishDate = item.get('establishDate')
            info['成立时间'] = str(datetime.datetime.fromtimestamp(establishDate/1000).date()) if establishDate else ''       # 成立时间有可能为空，防止这种情况而报错
            putOnRecordDate = item.get('putOnRecordDate')
            info['备案时间'] = str(datetime.datetime.fromtimestamp(putOnRecordDate/1000).date()) if putOnRecordDate else ''
            info['基金链接'] = fund_base_url + item.get('url')
            yield info

def write_to_json(lis):
    with open('info.json','w',encoding='utf-8') as f:
        json.dump({'info':lis},f,ensure_ascii=False)
        
def main():
    for page in range(1,100):
        r_json = get_page(page)
        results = parse_page(r_json)
        for result in results:
            lis_json.append(result)
    write_to_json(lis_json)

if __name__ == '__main__':
    main()


In [53]:
#爬中基协管理人数据
import requests
import random
import json
 
def save(school_datas):
    for data1 in school_datas:
        # print(data)
        id = data1['id']
        managerName = data1['managerName']
        artificialPersonName = data1['artificialPersonName']
        regAdrAgg = data1['regAdrAgg']
        registerNo = data1['registerNo']
        print(id, managerName, artificialPersonName, regAdrAgg,registerNo)
 
for i in range(0, 427):
    print("第%s页====================="%str(i))
    header={
            'Accept':'application/json, text/javascript, */*; q=0.01',
            'Accept-Encoding':'gzip, deflate',
            'Connection':'keep-alive',
            'Host':'gs.amac.org.cn',
            'Origin':'http://gs.amac.org.cn',
            'Referer':'http://gs.amac.org.cn/amac-infodisc/res/pof/manager/managerList.html',
            'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
            }
    r=random.random()
    print(str(r))
    
    # json={"rand":'0.0045470034372876444',"page":str(i),"size":"50"}
    # http://gs.amac.org.cn/amac-infodisc/api/pof/manager?rand=0.9775162173180119&page=1&size=50
    # data= requests.post("http://gs.amac.org.cn/amac-infodisc/api/pof/manager",json={'rand':str(r),'page':str(i),'size':'50'},headers=header)#.json()
    url="http://gs.amac.org.cn/amac-infodisc/api/pof/manager?rand=0.9775162173180119&page=%s&size=50"
    data= requests.post(url%i,json={'rand':str(r),'page':str(i),'size':'50'}).json()
  
    
    # print (type(r))
    # print (r.status_code)

    # print (r.cookies)
    # print(r.text,"\n")
    # print(r.json())


    print("每一页信息条数——>", len(data['content']))
    print("全部信息条数——>", data["totalElements"])
    print("每页有——>", data["size"])
    print("总页数-->>", data["totalPages"])

    school_datas = data["content"]
    save(school_datas)

第0页=====================
0.4868118159083338
每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000000138 平安道远投资管理（上海）有限公司 杨晓华 上海市 P1000182
101000000194 上海重阳投资管理股份有限公司 裘国根 上海市 P1000265
101000000211 上海景林资产管理有限公司 高云程 上海市 P1000267
101000000172 富舜资产管理（上海）有限公司 周楠 上海市 P1000268
101000000721 深圳民森投资有限公司 蔡明 深圳市 P1000269
101000000578 上海尚雅投资管理有限公司 石波 上海市 P1000271
101000000504 北京润晖资产管理有限公司 闵昱 北京市 P1000276
101000000580 新沃股权投资基金管理（天津）有限公司 朱灿 天津市 P1000287
101000000446 鼎晖股权投资管理（天津）有限公司 吴尚志 天津市 P1000301
101000000308 广东中科科创创业投资管理有限责任公司 杜昌焘 广东省 P1000302
101000000350 上海复星创富投资管理股份有限公司 唐斌 上海市 P1000303
101000000158 上海耀之资产管理中心（有限合伙） 王小坚 上海市 P1000318
101000000224 深圳东方港湾投资管理股份有限公司 但斌 深圳市 P1000320
101000000196 上海国富投资管理有限公司 陈海峰 上海市 P1000321
101000000530 上海世诚投资管理有限公司 陈家琳 上海市 P1000323
101000000292 深圳中睿合银投资管理有限公司 刘睿 深圳市 P1000326
101000000458 上海万丰友方投资管理有限公司 朱晓兵 上海市 P1000385
101000000195 深圳市翼虎投资管理有限公司 余定恒 深圳市 P1000388
101000000179 浙江思考投资集团股份有限公司 岳志武 浙江省 P1000391
101000000362 深圳市明曜投资管理有限公司 曾昭雄 深圳市 P1000406
1010000004

每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000001371 福建海峡文化产业股权投资管理有限公司 龚智坚 福建省 P1000745
101000000367 成都银科创业投资有限公司 吴忠 四川省 P1000746
101000002272 天津创业投资管理有限公司 李莉 天津市 P1000747
101000001546 航天科工投资基金管理（北京）有限公司 王洪民 北京市 P1000748
101000000539 苏州同源创业投资管理有限公司 邓锋 江苏省 P1000749
101000002073 光大控股创业投资（深圳）有限公司 殷连臣 深圳市 P1000750
101000000233 启明维创创业投资管理（北京）有限公司 邝子平 北京市 P1000751
101000000954 上海创丰创业投资管理有限公司 彭震 上海市 P1000752
101000001338 武汉高睿投资管理有限公司 邓维 湖北省 P1000753
101000000343 陕西盛德玖富投资管理企业（有限合伙） 陈军 陕西省 P1000754
101000001246 长沙联创永锋投资管理合伙企业（有限合伙） 苍凤华 湖南省 P1000755
101000001183 芜湖安益投资管理有限公司 杜龙泉 安徽省 P1000756
101000000931 北京南车创业投资有限公司 赵蔚 北京市 P1000757
101000000852 和谐爱奇投资管理（北京）有限公司 熊晓鸽 北京市 P1000758
101000001408 湖北高金投资管理有限公司 刘珂 湖北省 P1000759
101000001262 常州金茂经信创业投资管理企业(有限合伙) 段小光 江苏省 P1000760
101000001360 北京厚持投资管理有限责任公司 高鹏 北京市 P1000761
101000001278 芜湖兴安创业投资基金管理有限公司 牟坤林 安徽省 P1000762
101000001445 北京芳晟投资管理中心（有限合伙） 于淼 北京市 P1000763
101000002237 极地晨光创业投资管理（北京）有限公司 杨磊 北京市 P1000764
101000002110 悦达醴泉投资管理（上海）有限公司 朱

101000001588 宁波洲浚投资管理有限公司 许周军 宁波市 P1000893
101000001384 青岛赢隆资产管理有限公司 朱典 青岛市 P1000894
101000001212 上海培蓝投资管理有限公司 黄培蓝 上海市 P1000895
101000002136 上海大包巷投资管理有限公司 包周荣 上海市 P1000896
101000000943 上海丹寅投资管理中心（有限合伙） 高骐 上海市 P1000897
101000001437 北京天乙合资本管理有限公司 彭乃顺 北京市 P1000898
101000000398 国联产业投资基金管理（北京）有限公司 肖华 北京市 P1000899
101000002143 深圳市达晨财智创业投资管理有限公司 刘昼 深圳市 P1000900
101000000736 天津普拓股权投资基金管理有限公司 董群 天津市 P1000901
101000000338 芜湖歌斐资产管理有限公司 殷哲 安徽省 P1000902
101000000175 中非信银（上海）股权投资管理有限公司 孙亚雷 上海市 P1000903
101000000337 上海歌斐资产管理有限公司 殷哲 上海市 P1000904
101000000134 天津燕山投资管理有限公司 高建东 天津市 P1000905
101000002332 昆山京兆股权投资管理有限公司 万若谷 江苏省 P1000906
101000002234 天津裕丰股权投资管理有限公司 孔令芬 天津市 P1000907
101000001273 上海云锋投资管理有限公司 虞锋 上海市 P1000909
101000002779 上海谱润股权投资管理有限公司 尹锋 上海市 P1000910
101000003471 国开城镇商业发展基金管理（北京）有限公司 韦东政 北京市 P1000911
101000001116 稳盛（天津）投资管理有限公司 许亚峰 天津市 P1000912
101000002786 上海磐石元成投资有限公司 王力群 上海市 P1000913
101000001517 上海新德股权投资基金管理有限公司 贾力 上海市 P1000914
101000001698 北京金石农业投资基金管理中心（有限合伙） 杜雨辰 北京市 P1000915
101000001

101000002416 广东集成富达基金管理中心（有限合伙） 唐小凤 广东省 P1001119
101000003581 北京富坤中技投资管理有限公司 朱菁 北京市 P1001120
101000002904 上海壹德资产管理有限公司 韩长印 上海市 P1001122
101000001579 浙江钛和投资管理有限公司 潘晶 浙江省 P1001123
101000000460 上海联新投资咨询有限公司 曲列锋 上海市 P1001125
第12页=====================
0.1461925031282465
每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000002795 山东星展投资基金管理股份有限公司 张鲁 山东省 P1001126
101000003638 深圳市深商富坤兴业基金管理有限公司 林木雄 深圳市 P1001127
101000003654 首创方舟投资管理（北京）有限公司 刘晓光 北京市 P1001128
101000003971 上海歌石投资管理有限公司 何亚平 上海市 P1001129
101000000832 上海汉理股权投资管理股份有限公司 钱学锋 上海市 P1001130
101000003364 北京浙控金诚资产管理有限公司 赵锦明 北京市 P1001131
101000003389 苏州盛世宏明投资管理中心（有限合伙） 张洋 江苏省 P1001133
101000001694 北京启迪兴业创新投资有限公司 李晓 北京市 P1001134
101000002710 成都泰豪晟大股权投资基金管理中心（有限合伙） 李巍 四川省 P1001135
101000002393 华映光辉投资管理（苏州）有限公司 XIONG XIANGDONG 江苏省 P1001136
101000003551 天津文化产业股权投资基金管理有限公司 王树军 天津市 P1001137
101000001327 苏州农发创新资本管理有限公司 卢蕾 江苏省 P1001138
101000001717 成都华霖股权投资基金管理有限公司 舒绍敏 四川省 P1001139
101000002036 上海东胜股权投资有限公司 黄力进 上海市 P1001141
101000002398 常熟华映光辉投

101000002065 南京创动股权投资基金管理有限公司 袁冰 江苏省 P1001304
101000001638 加华伟业（天津）投资管理合伙企业（有限合伙） 宋向前 天津市 P1001305
101000001635 北京德尔泰达投资有限公司 孙红伟 北京市 P1001307
101000002248 仁和金砂（北京）投资基金管理有限公司 何鑫 北京市 P1001308
101000003013 乌鲁木齐聚源股权投资管理有限公司 吴兴华 新疆维吾尔自治区 P1001309
101000002401 华映资本管理有限公司 JI WEI 江苏省 P1001311
101000001071 建信天然(北京)投资顾问有限公司 袁圣尧 北京市 P1001313
101000003648 苏州长虞股权投资管理有限公司 黄庆伟 江苏省 P1001314
101000001544 益信开元（苏州）医疗健康投资管理有限公司 许萍 江苏省 P1001316
101000003875 中交投资基金管理（北京）有限公司 谢玉梅 北京市 P1001317
101000002266 西宁国家低碳产业基金投资管理有限公司 冯鹏 青海省 P1001319
101000002839 深圳市香江股权投资管理有限公司 刘根森 深圳市 P1001321
101000001439 建投华业（上海）股权投资管理有限公司 何文进 上海市 P1001324
第16页=====================
0.8462033175944934
每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000002341 德太沪华（上海）股权投资管理企业（有限合伙） Ronald Cami 上海市 P1001326
101000002550 德太庆华（重庆）股权投资管理合伙企业（有限合伙） Ronald Cami 重庆市 P1001327
101000002583 德太海华（上海）股权投资管理有限公司 黄卓群 上海市 P1001328
101000003406 中铁建信（北京）投资基金管理有限公司 汪涛 北京市 P1001329
101000002253 宁波信达汉石投资管理有限公司 宋铭贺 宁波市 P1001334
101000004380 宁波鼎锋海川

...

每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000028026 贵州众石银杉资本管理有限公司 李亮 贵州省 P1019604
101000028532 广州银易多私募证券投资管理有限公司 杨虹 广东省 P1019605
101000029113 山东银企投资发展股份有限公司 林如意 山东省 P1019606
101000029199 深圳丰之银股权基金管理有限公司 冷立昌 深圳市 P1019608
101000028627 苏州天红嘉华创业投资合伙企业（有限合伙） 周剑江 江苏省 P1019609
101000025798 深圳市翊丰投资有限公司 宋应 深圳市 P1019611
101000023731 上海升展投资管理有限公司 姜志浩 上海市 P1019617
101000028870 深圳前海三维同创基金管理有限公司 曾燕玲 深圳市 P1019620
101000028959 上海莫非投资管理有限公司 朱燕 上海市 P1019629
101000029562 南京金伯珠资产管理有限公司 后江帆 江苏省 P1019631
101000026319 北京划锌拢辛股权投资基金管理有限公司 赵英凯 北京市 P1019637
101000028513 上海王彬资产管理有限公司 王彬 上海市 P1019639
101000029149 华盖卓信投资管理（北京）有限公司 鹿炳辉 北京市 P1019641
101000029266 华盖创业投资管理（北京）有限公司 鹿炳辉 北京市 P1019643
101000027940 珠海横琴粤信资产管理有限公司 刘万锋 广东省 P1019644
101000028068 深圳前海昆桐资产管理有限公司 黄毅忠 深圳市 P1019648
101000028240 深圳市上古投资管理有限公司 王伟 深圳市 P1019649
101000027193 武汉每天不一样投资有限公司 李辉 湖北省 P1019652
101000029580 深圳市鹰涧资产管理有限公司 唐越 深圳市 P1019653
101000029099 北京华宏财富投资管理有限公司 班华全 北京市 P1019654
101000027830 深圳易观长河基金管理有限公司 解小琦 深圳市 P1019655
1010

101000020444 杭州厚华投资管理有限公司 严文伟 浙江省 P1020240
101000028712 上海荣巽资产管理中心（有限合伙） 杨梅 上海市 P1020243
101000015416 嘉兴溥原股权投资管理有限公司 伊继明 浙江省 P1020246
101000011092 上海鹰石股权投资管理有限公司 高汕 上海市 P1020247
101000029070 深圳德丰嘉禾基金管理有限公司 刘长征 深圳市 P1020249
101000028369 深圳瑞福资产管理有限公司 温东平 深圳市 P1020251
101000030074 上海盈嘉信投资管理合伙企业（有限合伙） 杨宁 上海市 P1020253
101000030153 中传平安（北京）投资基金管理有限公司 王垚 北京市 P1020254
101000030396 国投信邦（北京）资产管理有限公司 翟德元 北京市 P1020257
101000030159 上海风蓝资产管理有限公司 蓝海平 上海市 P1020259
101000029154 北京聚信达基金管理有限公司 李和霖 北京市 P1020265
101000027751 珠海市横琴志合股权投资管理合伙企业（有限合伙） 李政麟 广东省 P1020269
101000030238 浙江澳创资产管理有限公司 孙玉涛 浙江省 P1020279
101000028640 山东招金投资股份有限公司 李宜三 山东省 P1020281
101000029401 江西世行大通资产管理有限公司 吕灵斐 江西省 P1020282
101000030350 国投汇金资产管理（深圳）有限公司 侯颂 深圳市 P1020283
101000027117 上海融国投资管理有限公司 陈奔特 上海市 P1020286
101000026898 上海晨澳股权投资管理有限公司 宋佩荣 上海市 P1020287
101000028582 北京东宏世家资产管理有限公司 王东亮 北京市 P1020293
101000030012 上海湖杉投资管理有限公司 苏仁宏 上海市 P1020294
101000030202 深圳好方得投资有限公司 刘晓芳 深圳市 P1020297
101000030359 上海晟犀投资管理合伙企业（有限合伙） 陈琦琳 上海市 P1020299
10100

每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000031380 深圳市美丽图画资产管理有限公司 孙红艳 深圳市 P1020947
101000031058 北京万方鑫润基金管理有限公司 刘玉 北京市 P1020950
101000031342 宁波汪洋浩博资产管理中心（有限合伙） 陈海钢 宁波市 P1020951
101000030823 宁波凯盈投资管理有限公司 周炜 宁波市 P1020957
101000030846 江苏金珊瑚资本管理有限公司 杨昊佳 江苏省 P1020958
101000026834 华盖南方投资管理（深圳）有限公司 韦韧 深圳市 P1020959
101000027990 北京乾创投资管理有限公司 陈臻 北京市 P1020960
101000031602 北京国投创盈投资管理有限公司 王宇 北京市 P1020962
101000031383 深圳前海中瑞达资产管理有限公司 王予宾 深圳市 P1020966
101000014638 深圳华厦财富基金管理有限公司 由云峰 深圳市 P1020971
101000031350 天玑创投（北京）投资基金管理有限公司 王臻 北京市 P1020976
101000030523 上海云炜资产管理有限公司 葛亚莎 上海市 P1020977
101000026124 上海巴奇索投资管理有限公司 芦维雄 上海市 P1020979
101000009661 福建省磊鼎资产管理有限公司 林千翔 厦门市 P1020987
101000031033 弘轩永盛（北京）投资基金管理有限公司 彭仕行 北京市 P1020988
101000025995 深圳丰盈传褀资产管理有限公司 姜洪文 深圳市 P1020993
101000022958 甲午源石资本管理（北京）有限公司 杨凡 北京市 P1020994
101000031199 北京融达财富投资咨询有限公司 陈志文 北京市 P1021004
101000030812 杭州坤石达宝投资管理有限公司 石敏军 浙江省 P1021006
101000031512 浙江秘银投资管理有限公司 钱哲 浙江省 P1021014
101000031404 信合财富（北京）基金管理有限公司 石双月 北京市 P1021017
10

每一页信息条数——> 50
全部信息条数——> 24446
每页有——> 50
总页数-->> 489
101000031741 上海根鸟资产管理有限公司 沈小华 上海市 P1021571
101000032035 深圳前海正道启迪资产管理有限公司 宋屹 深圳市 P1021574
101000032115 无锡易君甲号股权投资中心（有限合伙） 沈钧 江苏省 P1021575
101000032112 深圳前海红宝石创投基金管理有限公司 卢峥 深圳市 P1021576
101000032083 北京文资虚苑艺术品投资管理有限公司 王颢隶 北京市 P1021578
101000031299 成都弘阳股权投资基金管理有限公司 史少博 四川省 P1021579
101000028302 天津恒石资产管理有限公司 寇永红 天津市 P1021583
101000031149 北京驭风青云投资管理有限公司 李克登 北京市 P1021584
101000017788 广州市乾元资产管理有限公司 詹东征 广东省 P1021586
101000030798 北京厚毅资本管理有限公司 王又红 北京市 P1021590
101000030358 智慧生活投资管理河北有限公司 康健 河北省 P1021591
101000022600 上海脉尊股权投资基金管理有限公司 郑世龙 上海市 P1021592
101000021784 杭州巨鲸财富管理有限公司 倪心刚 浙江省 P1021593
101000028864 广东普阳资产管理有限公司 林立容 广东省 P1021594
101000029324 北京泓赢资本投资管理有限公司 高可珈 北京市 P1021595
101000029143 北京盈帆资产管理有限公司 王小葵 北京市 P1021596
101000031592 深圳前海千灯恒力资产管理有限公司 杨浩 深圳市 P1021597
101000031813 深圳市桐谷资产管理有限公司 余培珍 深圳市 P1021599
101000032178 上海惠昌股权投资基金管理有限公司 陶琳 上海市 P1021600
101000032123 深圳同佳投资基金管理有限公司 兰志龙 深圳市 P1021601
101000031458 文品阁（深圳）金融科技有限公司 谢克祥 深圳市 P1021602
10

KeyboardInterrupt: 

In [199]:
#自行调试
from urllib.parse import urlencode
import requests
import json
import datetime

base_url = 'http://gs.amac.org.cn/amac-infodisc/api/pof/fund?' 

#'http://gs.amac.org.cn/amac-infodisc/api/pof/fund?rand=0.7438677869730821&page=2&size=20'


headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '2',
    'Content-Type': 'application/json',
    'Host': 'gs.amac.org.cn',
    'Origin': 'http://gs.amac.org.cn',
    'Referer': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/index.html',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}


params = {                                                  
        'rand': 0.7438677869730821,
        'page': 5185,
        'size': 20
    }

url = base_url + urlencode(params)

print(url)
    
r = requests.post(url, headers = headers, data = json.dumps({})).json()

print(type(r))
#print(r.get('content'))
items = r.get('content')
for item in items:
    info = {}
    info['基金名称'] = item.get('fundName')
    info['私募基金管理人名称'] = item.get('managerName')
    info['托管人名称'] = item.get('mandatorName')
    establishDate = item.get('establishDate')
    info['成立时间'] = str(datetime.datetime.fromtimestamp(establishDate/1000).date()) if establishDate else ''       # 成立时间有可能为空，防止这种情况而报错
    putOnRecordDate = item.get('putOnRecordDate')
    info['备案时间'] = str(datetime.datetime.fromtimestamp(putOnRecordDate/1000).date()) if putOnRecordDate else ''
    info['基金链接'] = fund_base_url + item.get('url')
    print(info)



http://gs.amac.org.cn/amac-infodisc/api/pof/fund?rand=0.7438677869730821&page=5185&size=20
<class 'dict'>
{'基金名称': '西咸新区清源创业投资合伙企业（有限合伙）', '私募基金管理人名称': '海南清源创新股权投资基金管理有限公司', '托管人名称': '中国建设银行股份有限公司', '成立时间': '2018-12-19', '备案时间': '2019-01-09', '基金链接': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/1901091212107144.html'}
{'基金名称': '天演大有天算2号私募证券投资基金', '私募基金管理人名称': '天算量化（北京）资本管理有限公司', '托管人名称': '中信证券股份有限公司', '成立时间': '2018-12-28', '备案时间': '2019-01-09', '基金链接': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/1901091212106232.html'}
{'基金名称': '深圳顺利钱投资企业（有限合伙）', '私募基金管理人名称': '杭州简朴投资管理有限公司', '托管人名称': '', '成立时间': '2018-12-20', '备案时间': '2019-01-09', '基金链接': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/1901091212107357.html'}
{'基金名称': '漳州招商局经济技术开发区合泽股权投资合伙企业（有限合伙）', '私募基金管理人名称': '宁波梅山保税港区魔范投资管理有限责任公司', '托管人名称': '', '成立时间': '2018-12-06', '备案时间': '2019-01-09', '基金链接': 'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/1901091212107267.html'}
{'基金名称': '上海百晨创业投资中心(有限合伙)', '私募基金管理人名称': '上海晨晖创业投资